In [1]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, LSTM, Flatten, Reshape, ZeroPadding1D, Conv1D, MaxPooling1D, UpSampling1D, Cropping1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K
import gc
import tensorflow as tf
import random
from sklearn import preprocessing
import pickle
from utilities import *

In [2]:
#Load config
config = loadData("config")
dataSetSize = config["dataSetSize"]
testSetBenignSize = config["testSetBenignSize"]
validationSetSize = config["validationSetSize"]
trainingSetSize = config["trainingSetSize"]
sequenceLen = config["sequenceLen"]
dimensionsCount = config["dimensionsCount"]
attackSamplesCount = config["attackSamplesCount"]
testSetSize = testSetBenignSize + attackSamplesCount

In [3]:
dataSet = loadData("normalized_data_set")
advDataSet = loadData("adversarial_data_set_lstm")

### PCA

In [4]:
def trainPCA(codeSize, trainingSetAdv, trainingSetLabels, validationSet, printSummary = 1, vrbs = 1, return_best = 0, learningRate = 0.001):
    inputSize = sequenceLen * dimensionsCount
    model = Sequential()
    model.add(Dense(inputSize, input_shape=(inputSize,), activation='linear'))
    model.add(Dense(int(codeSize), activation="linear"))
    model.add(Dense(inputSize, activation="linear"))
    if(printSummary == True):
            model.summary()
    adamOptimizer = Adam(learning_rate=learningRate) # , beta_1=0.9, beta_2=0.999, amsgrad=False        
    model.compile(optimizer=adamOptimizer,
        loss='mean_squared_error',
    )
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 10)
    callbacksArray = [es]
    if(return_best):
        mc = ModelCheckpoint('best_pca.h5', monitor='val_loss', mode='min')
        callbacksArray = [es, mc]
    
    trainingSet, testSet, validationSet = getReshapedDataSet(dataSet, "PCA")
    history=model.fit(trainingSet, trainingSet,
                            batch_size=5000,
                            shuffle=True,
                            epochs=10000,                             
                            validation_data=(validationSet, validationSet),
                            callbacks=callbacksArray,
                            verbose = vrbs,
                        )
    
    if(return_best):
        best_model = load_model('best_pca.h5')
        
    returnModel = model
    if(return_best):
        returnModel = best_model
    return [returnModel,min(history.history['val_loss']),len(history.history['val_loss']),history]

In [5]:
modelType = "PCA"
trainingSetAdv, trainingSetLabels, testSet, validationSet = getReshapedAdversarialDataSet(dataSet, advDataSet, modelType)
codeLayerSize = 50
[model, validatoinLoss, numOfEpochs,_] = trainPCA(codeLayerSize, trainingSetAdv, trainingSetLabels, validationSet)
model.save('Trained_Model/pca_adv.h5')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1800)              3241800   
_________________________________________________________________
dense_1 (Dense)              (None, 50)                90050     
_________________________________________________________________
dense_2 (Dense)              (None, 1800)              91800     
Total params: 3,423,650
Trainable params: 3,423,650
Non-trainable params: 0
_________________________________________________________________
Train on 11060 samples, validate on 1382 samples
Epoch 1/10000
11060/11060 [==============================] - 1s 67us/sample - loss: 0.1090 - val_loss: 0.0240
Epoch 2/10000
11060/11060 [==============================] - 0s 12us/sample - loss: 0.0758 - val_loss: 0.0238
Epoch 3/10000
11060/11060 [==============================] - 0s 12us/sample - loss: 0.0630 - val_loss: 0.02

11060/11060 [==============================] - 0s 11us/sample - loss: 8.1005e-04 - val_loss: 0.0028
Epoch 68/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 7.9182e-04 - val_loss: 0.0028
Epoch 69/10000
11060/11060 [==============================] - 0s 13us/sample - loss: 7.6420e-04 - val_loss: 0.0027
Epoch 70/10000
11060/11060 [==============================] - 0s 12us/sample - loss: 7.6099e-04 - val_loss: 0.0026
Epoch 71/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 7.5997e-04 - val_loss: 0.0025
Epoch 72/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 7.7235e-04 - val_loss: 0.0025
Epoch 73/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 8.3968e-04 - val_loss: 0.0024
Epoch 74/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 9.0133e-04 - val_loss: 0.0023
Epoch 75/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 7.4779e-04

11060/11060 [==============================] - 0s 11us/sample - loss: 4.4049e-04 - val_loss: 0.0011
Epoch 139/10000
11060/11060 [==============================] - 0s 14us/sample - loss: 5.5561e-04 - val_loss: 0.0011
Epoch 140/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 7.4092e-04 - val_loss: 0.0011
Epoch 141/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 5.1088e-04 - val_loss: 0.0011
Epoch 142/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 5.6715e-04 - val_loss: 0.0011
Epoch 143/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 4.6354e-04 - val_loss: 0.0011
Epoch 144/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 4.1863e-04 - val_loss: 0.0011
Epoch 145/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 3.9693e-04 - val_loss: 0.0010
Epoch 146/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 3.

11060/11060 [==============================] - 0s 11us/sample - loss: 2.8170e-04 - val_loss: 8.7297e-04
Epoch 208/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 2.8001e-04 - val_loss: 8.7127e-04
Epoch 209/10000
11060/11060 [==============================] - 0s 12us/sample - loss: 2.7978e-04 - val_loss: 8.6881e-04
Epoch 210/10000
11060/11060 [==============================] - 0s 13us/sample - loss: 2.8293e-04 - val_loss: 8.6585e-04
Epoch 211/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 2.9220e-04 - val_loss: 8.6540e-04
Epoch 212/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 3.2473e-04 - val_loss: 8.6319e-04
Epoch 213/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 3.8452e-04 - val_loss: 8.7407e-04
Epoch 214/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 3.8192e-04 - val_loss: 8.6251e-04
Epoch 215/10000
11060/11060 [===========================

11060/11060 [==============================] - 0s 10us/sample - loss: 2.6223e-04 - val_loss: 7.5221e-04
Epoch 276/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 2.5043e-04 - val_loss: 7.5014e-04
Epoch 277/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 2.5026e-04 - val_loss: 7.4781e-04
Epoch 278/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 2.4350e-04 - val_loss: 7.4725e-04
Epoch 279/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 2.4200e-04 - val_loss: 7.4413e-04
Epoch 280/10000
11060/11060 [==============================] - 0s 15us/sample - loss: 2.3960e-04 - val_loss: 7.4218e-04
Epoch 281/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 2.3876e-04 - val_loss: 7.3876e-04
Epoch 282/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 2.3636e-04 - val_loss: 7.3691e-04
Epoch 283/10000
11060/11060 [===========================

11060/11060 [==============================] - 0s 11us/sample - loss: 2.3705e-04 - val_loss: 6.1542e-04
Epoch 344/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 2.5354e-04 - val_loss: 6.1154e-04
Epoch 345/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 2.7380e-04 - val_loss: 6.1621e-04
Epoch 346/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 2.8329e-04 - val_loss: 6.1233e-04
Epoch 347/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 3.4833e-04 - val_loss: 6.1740e-04
Epoch 348/10000
11060/11060 [==============================] - 0s 12us/sample - loss: 3.2073e-04 - val_loss: 6.0231e-04
Epoch 349/10000
11060/11060 [==============================] - 0s 14us/sample - loss: 2.8363e-04 - val_loss: 6.0706e-04
Epoch 350/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 2.2901e-04 - val_loss: 5.9945e-04
Epoch 351/10000
11060/11060 [===========================

11060/11060 [==============================] - 0s 11us/sample - loss: 1.7282e-04 - val_loss: 4.9186e-04
Epoch 412/10000
11060/11060 [==============================] - 0s 12us/sample - loss: 1.9524e-04 - val_loss: 4.9141e-04
Epoch 413/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 1.7148e-04 - val_loss: 4.9087e-04
Epoch 414/10000
11060/11060 [==============================] - 0s 12us/sample - loss: 1.6901e-04 - val_loss: 4.8679e-04
Epoch 415/10000
11060/11060 [==============================] - 0s 12us/sample - loss: 1.7121e-04 - val_loss: 4.8614e-04
Epoch 416/10000
11060/11060 [==============================] - 0s 12us/sample - loss: 1.7632e-04 - val_loss: 4.8467e-04
Epoch 417/10000
11060/11060 [==============================] - 0s 14us/sample - loss: 1.8828e-04 - val_loss: 4.8873e-04
Epoch 418/10000
11060/11060 [==============================] - 0s 14us/sample - loss: 2.2684e-04 - val_loss: 4.8637e-04
Epoch 419/10000
11060/11060 [===========================

11060/11060 [==============================] - 0s 15us/sample - loss: 1.6746e-04 - val_loss: 4.1561e-04
Epoch 480/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 2.0657e-04 - val_loss: 4.1101e-04
Epoch 481/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 2.3659e-04 - val_loss: 4.1628e-04
Epoch 482/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 1.6562e-04 - val_loss: 4.1013e-04
Epoch 483/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 1.8109e-04 - val_loss: 4.0789e-04
Epoch 484/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 1.9177e-04 - val_loss: 4.1069e-04
Epoch 485/10000
11060/11060 [==============================] - 0s 11us/sample - loss: 1.9932e-04 - val_loss: 4.0697e-04
Epoch 486/10000
11060/11060 [==============================] - 0s 14us/sample - loss: 1.6191e-04 - val_loss: 4.0650e-04
Epoch 487/10000
11060/11060 [===========================

### Fully Connected

In [6]:
def trainAutoencoder(numOfHiddenLayersInEncoder, codeLayerSize, trainingSet, labels, validationSet, printSummary = 1, vrbs = 1, return_best = 1, learningRate = 0.001):
    inputSize = sequenceLen * dimensionsCount
    layerSizeDifference = (inputSize - codeLayerSize) // (numOfHiddenLayersInEncoder + 1)
    model = Sequential()
    
    print(layerSizeDifference)
    
    
    #NeuronsCountInFirstLayer = 64
    model.add(Dense(inputSize, input_shape=(inputSize,), activation='relu'))
    for i in range(1,numOfHiddenLayersInEncoder + 1):
        model.add(Dense(int(inputSize - (layerSizeDifference * i)), activation="relu"))
        
    model.add(Dense(int(codeLayerSize), activation="relu"))
        
    for j in range(1,numOfHiddenLayersInEncoder + 1):    
        model.add(Dense(int(inputSize - (layerSizeDifference * (numOfHiddenLayersInEncoder - j + 1))), activation="relu"))
        
    model.add(Dense(inputSize, activation="linear"))

    if(printSummary == True):
            model.summary()

    adamOptimizer = Adam(learning_rate=learningRate) # , beta_1=0.9, beta_2=0.999, amsgrad=False        
            
    model.compile(optimizer=adamOptimizer,
        loss='mean_squared_error',
    )
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 10)
    callbacksArray = [es]
    if(return_best):
        mc = ModelCheckpoint('best_Autoencoder.h5', monitor='val_loss', mode='min')
        callbacksArray = [es, mc]
    
    history=model.fit(trainingSet, labels,
                            batch_size=5000,
                            shuffle=True,
                            epochs=10000,                             
                            validation_data=(validationSet, validationSet),
                            callbacks=callbacksArray,
                            verbose = vrbs,
                        )
    
    if(return_best):
        best_model = load_model('best_Autoencoder.h5')
        
    returnModel = model
    if(return_best):
        returnModel = best_model
    return [returnModel,min(history.history['val_loss']),len(history.history['val_loss']),history]

In [7]:
modelType = "fullyConnected"
trainingSetAdv, trainingSetLabels, testSet, validationSet = getReshapedAdversarialDataSet(dataSet, advDataSet, modelType)
numOfHiddenLayers = 1
codeLayerSize = 40 # 64
[model, validatoinLoss, numOfEpochs, history] = trainAutoencoder(numOfHiddenLayers, codeLayerSize, trainingSetAdv, trainingSetLabels, validationSet, return_best = 1)
model.save('Trained_Model/autoencoder_adv.h5')

880
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 1800)              3241800   
_________________________________________________________________
dense_4 (Dense)              (None, 920)               1656920   
_________________________________________________________________
dense_5 (Dense)              (None, 40)                36840     
_________________________________________________________________
dense_6 (Dense)              (None, 920)               37720     
_________________________________________________________________
dense_7 (Dense)              (None, 1800)              1657800   
Total params: 6,631,080
Trainable params: 6,631,080
Non-trainable params: 0
_________________________________________________________________
Train on 11310 samples, validate on 1382 samples
Epoch 1/10000
11310/11310 [==============================

Epoch 63/10000
11310/11310 [==============================] - 0s 39us/sample - loss: 2.4161e-04 - val_loss: 6.4596e-04
Epoch 64/10000
11310/11310 [==============================] - 1s 49us/sample - loss: 2.3914e-04 - val_loss: 6.4198e-04
Epoch 65/10000
11310/11310 [==============================] - 0s 32us/sample - loss: 2.3643e-04 - val_loss: 6.3664e-04
Epoch 66/10000
11310/11310 [==============================] - 0s 34us/sample - loss: 2.3355e-04 - val_loss: 6.2842e-04
Epoch 67/10000
11310/11310 [==============================] - 1s 45us/sample - loss: 2.3111e-04 - val_loss: 6.2808e-04
Epoch 68/10000
11310/11310 [==============================] - 0s 34us/sample - loss: 2.2922e-04 - val_loss: 6.2030e-04
Epoch 69/10000
11310/11310 [==============================] - 0s 35us/sample - loss: 2.2761e-04 - val_loss: 6.1745e-04
Epoch 70/10000
11310/11310 [==============================] - 0s 44us/sample - loss: 2.2530e-04 - val_loss: 6.1282e-04
Epoch 71/10000
11310/11310 [====================

11310/11310 [==============================] - 0s 36us/sample - loss: 1.2156e-04 - val_loss: 3.8630e-04
Epoch 132/10000
11310/11310 [==============================] - 0s 42us/sample - loss: 1.2067e-04 - val_loss: 3.8613e-04
Epoch 133/10000
11310/11310 [==============================] - 0s 34us/sample - loss: 1.2064e-04 - val_loss: 3.8259e-04
Epoch 134/10000
11310/11310 [==============================] - 0s 36us/sample - loss: 1.1849e-04 - val_loss: 3.7832e-04
Epoch 135/10000
11310/11310 [==============================] - 0s 36us/sample - loss: 1.1749e-04 - val_loss: 3.7680e-04
Epoch 136/10000
11310/11310 [==============================] - 0s 32us/sample - loss: 1.1648e-04 - val_loss: 3.7332e-04
Epoch 137/10000
11310/11310 [==============================] - 0s 34us/sample - loss: 1.1655e-04 - val_loss: 3.7472e-04
Epoch 138/10000
11310/11310 [==============================] - 1s 48us/sample - loss: 1.2070e-04 - val_loss: 3.8560e-04
Epoch 139/10000
11310/11310 [===========================

### 1D Conv

In [8]:
def train1DConv(numOfHiddenLayersInEncoder, trainingSet, labels, validationSet, FiltersCountInFirstLayer = 32, printSummary = 1, vrbs = 1, return_best = 1, filterSize = 3, learningRate = 0.001):
    numOfHiddenLayersInEncoder = numOfHiddenLayersInEncoder + 1
    model = Sequential()    
    poolingSize = 2
    numOfFiltersInEncoder = [FiltersCountInFirstLayer]
    paddingSize = (filterSize // 2)
    inputLen = (sequenceLen * dimensionsCount)
    paddedInputLength = inputLen + (paddingSize*2)
    encoderLayersFilterSizes = [paddedInputLength]
    model.add(ZeroPadding1D(paddingSize,input_shape=(sequenceLen * dimensionsCount,1)))
    model.add(Conv1D(int(FiltersCountInFirstLayer), filterSize, activation='relu', padding = 'same'))
    for i in range(1,numOfHiddenLayersInEncoder):
        model.add(MaxPooling1D(poolingSize, padding='same'))
        model.add(Conv1D(int(FiltersCountInFirstLayer*np.power(2,i)), filterSize, activation='relu', padding = 'same'))
        
        encoderLayersFilterSizes.append(int(np.ceil(paddedInputLength/np.power(2,i))))
        numOfFiltersInEncoder.append(int(FiltersCountInFirstLayer*np.power(2,i)))
    model.add(Flatten())
    model.add(Reshape((encoderLayersFilterSizes[-1], numOfFiltersInEncoder[-1])))
    for j in range(1,numOfHiddenLayersInEncoder):    
        model.add(Conv1D(numOfFiltersInEncoder[-j], filterSize, activation='relu', padding = 'same'))
        model.add(UpSampling1D(poolingSize))

    model.add(Conv1D(numOfFiltersInEncoder[0], filterSize, activation='relu', padding='same'))
    model.add(Conv1D(1, filterSize, activation='linear', padding='same'))
    
    toCrop = (model.layers[-1].output_shape[1] - inputLen) // 2
    
    model.add(Cropping1D(toCrop))
    model.summary()

    adamOptimizer = Adam(learning_rate=learningRate) # , beta_1=0.9, beta_2=0.999, amsgrad=False            
    model.compile(optimizer=adamOptimizer,
        loss='mean_squared_error',
    )

    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 10)
    callbacksArray = [es]
    if(return_best):
        mc = ModelCheckpoint('best_1DConv.h5', monitor='val_loss', mode='min')
        callbacksArray = [es, mc]
   
    history=model.fit(trainingSet, labels,
                            batch_size=1000,
                            shuffle=True,
                            epochs=10000,                             
                            validation_data=(validationSet, validationSet),
                            callbacks=callbacksArray,
                            verbose = vrbs,
                        )
    
    if(return_best):
        best_model = load_model('best_1DConv.h5')
        
    returnModel = model
    if(return_best):
        returnModel = best_model
    return [returnModel,min(history.history['val_loss']),len(history.history['val_loss']),history]

In [9]:
modelType = "1DConv"
trainingSetAdv, trainingSetLabels, testSet, validationSet = getReshapedAdversarialDataSet(dataSet, advDataSet, modelType)
numOfLayers = 2
filtersCountInFirstLayer = 48
[model, validatoinLoss, numOfEpochs, history] = train1DConv(numOfLayers, trainingSetAdv, trainingSetLabels, validationSet, filtersCountInFirstLayer, filterSize = 4)
model.save('Trained_Model/conv_adv.h5')

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding1d (ZeroPadding1 (None, 1804, 1)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 1804, 48)          240       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 902, 48)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 902, 96)           18528     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 451, 96)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 451, 192)          73920     
_________________________________________________________________
flatten (Flatten)            (None, 86592)            

11310/11310 [==============================] - 4s 371us/sample - loss: 7.9412e-05 - val_loss: 2.5618e-04
Epoch 51/10000
11310/11310 [==============================] - 4s 361us/sample - loss: 1.1246e-04 - val_loss: 2.4358e-04
Epoch 52/10000
11310/11310 [==============================] - 4s 362us/sample - loss: 8.7322e-05 - val_loss: 2.2384e-04
Epoch 53/10000
11310/11310 [==============================] - 4s 362us/sample - loss: 7.4016e-05 - val_loss: 2.2100e-04
Epoch 54/10000
11310/11310 [==============================] - 4s 364us/sample - loss: 6.9202e-05 - val_loss: 2.1535e-04
Epoch 55/10000
11310/11310 [==============================] - 4s 362us/sample - loss: 8.9803e-05 - val_loss: 2.2264e-04
Epoch 56/10000
11310/11310 [==============================] - 4s 362us/sample - loss: 9.0424e-05 - val_loss: 2.2752e-04
Epoch 57/10000
11310/11310 [==============================] - 4s 362us/sample - loss: 6.6243e-05 - val_loss: 2.1840e-04
Epoch 58/10000
11310/11310 [===========================

11310/11310 [==============================] - 4s 354us/sample - loss: 1.9968e-05 - val_loss: 9.2965e-05
Epoch 119/10000
11310/11310 [==============================] - 4s 345us/sample - loss: 1.9154e-05 - val_loss: 9.5369e-05
Epoch 120/10000
11310/11310 [==============================] - 4s 373us/sample - loss: 4.7388e-05 - val_loss: 2.0921e-04
Epoch 121/10000
11310/11310 [==============================] - 4s 364us/sample - loss: 1.4395e-04 - val_loss: 1.2889e-04
Epoch 122/10000
11310/11310 [==============================] - 4s 364us/sample - loss: 4.4619e-05 - val_loss: 9.9691e-05
Epoch 123/10000
11310/11310 [==============================] - 4s 363us/sample - loss: 2.7127e-05 - val_loss: 9.2398e-05
Epoch 124/10000
11310/11310 [==============================] - 4s 363us/sample - loss: 2.1321e-05 - val_loss: 8.9324e-05
Epoch 125/10000
11310/11310 [==============================] - 4s 364us/sample - loss: 1.8814e-05 - val_loss: 9.2320e-05
Epoch 126/10000
11310/11310 [===================

Epoch 186/10000
11310/11310 [==============================] - 4s 345us/sample - loss: 1.3371e-05 - val_loss: 5.2862e-05
Epoch 187/10000
11310/11310 [==============================] - 4s 344us/sample - loss: 1.1595e-05 - val_loss: 5.2183e-05
Epoch 188/10000
11310/11310 [==============================] - 4s 351us/sample - loss: 1.1107e-05 - val_loss: 5.2143e-05
Epoch 189/10000
11310/11310 [==============================] - 4s 350us/sample - loss: 1.3617e-05 - val_loss: 5.2076e-05
Epoch 190/10000
11310/11310 [==============================] - 4s 347us/sample - loss: 1.5693e-05 - val_loss: 5.2383e-05
Epoch 191/10000
11310/11310 [==============================] - 4s 368us/sample - loss: 4.8739e-05 - val_loss: 8.5897e-05
Epoch 192/10000
11310/11310 [==============================] - 4s 362us/sample - loss: 3.4915e-05 - val_loss: 5.6685e-05
Epoch 193/10000
11310/11310 [==============================] - 4s 369us/sample - loss: 1.8803e-05 - val_loss: 5.1356e-05
Epoch 194/10000
11310/11310 [===

11310/11310 [==============================] - 4s 368us/sample - loss: 1.3070e-05 - val_loss: 3.2258e-05
Epoch 254/10000
11310/11310 [==============================] - 4s 367us/sample - loss: 1.0039e-05 - val_loss: 3.2411e-05
Epoch 255/10000
11310/11310 [==============================] - 4s 371us/sample - loss: 1.0211e-05 - val_loss: 3.8677e-05
Epoch 256/10000
11310/11310 [==============================] - 4s 366us/sample - loss: 2.9214e-05 - val_loss: 3.3522e-05
Epoch 257/10000
11310/11310 [==============================] - 4s 366us/sample - loss: 1.3447e-05 - val_loss: 3.3036e-05
Epoch 258/10000
11310/11310 [==============================] - 4s 375us/sample - loss: 1.1349e-05 - val_loss: 3.1616e-05
Epoch 259/10000
11310/11310 [==============================] - 4s 368us/sample - loss: 8.8834e-06 - val_loss: 3.5058e-05
Epoch 260/10000
11310/11310 [==============================] - 4s 367us/sample - loss: 1.5335e-05 - val_loss: 3.9949e-05
Epoch 261/10000
11310/11310 [===================

Epoch 321/10000
11310/11310 [==============================] - 4s 367us/sample - loss: 7.2762e-06 - val_loss: 2.5779e-05
Epoch 322/10000
11310/11310 [==============================] - 4s 368us/sample - loss: 1.2672e-05 - val_loss: 4.5353e-05
Epoch 323/10000
11310/11310 [==============================] - 4s 368us/sample - loss: 3.7008e-05 - val_loss: 2.8532e-05
Epoch 324/10000
11310/11310 [==============================] - 4s 371us/sample - loss: 1.2092e-05 - val_loss: 2.3751e-05
Epoch 325/10000
11310/11310 [==============================] - 4s 368us/sample - loss: 7.2297e-06 - val_loss: 2.3455e-05
Epoch 326/10000
11310/11310 [==============================] - 4s 369us/sample - loss: 6.0876e-06 - val_loss: 2.2878e-05
Epoch 327/10000
11310/11310 [==============================] - 4s 385us/sample - loss: 5.6109e-06 - val_loss: 2.2620e-05
Epoch 328/10000
11310/11310 [==============================] - 4s 369us/sample - loss: 6.5238e-06 - val_loss: 2.8948e-05
Epoch 329/10000
11310/11310 [===

### LSTM 

In [10]:
#function changed
def trainLSTMAdv(numOfLayers, numOfNeurons, trainingSet, labels, validationSet, printSummary = 1, vrbs = 1, return_best = 1, learningRate = 0.001):
        sequenceLen = dataSet.shape[1]
        dimensionsCount = dataSet.shape[2]
        
        print('Training model.')
        model = Sequential()
        model.add(LSTM(numOfNeurons, input_shape=(sequenceLen, dimensionsCount), return_sequences=True))
        for j in range(0, numOfLayers - 1):
            model.add(LSTM(numOfNeurons, return_sequences=True))

        model.add(Dense(dimensionsCount))
        model.add(Activation("linear"))

        if(printSummary == True):
            model.summary()

        adamOptimizer = Adam(learning_rate=learningRate) # , beta_1=0.9, beta_2=0.999, amsgrad=False            
        model.compile(optimizer=adamOptimizer,
            loss='mean_squared_error',
        )

        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 10)
        callbacksArray = [es]
        if(return_best):
            mc = ModelCheckpoint('best_lstm.h5', monitor='val_loss', mode='min')
            callbacksArray = [es, mc]

        history=model.fit(trainingSet, labels,
                            batch_size=5000,
                            shuffle=True,
                            epochs=10000, 
                            #validation_split=0.1,
                            validation_data=(validationSet, validationSet),
                            callbacks=callbacksArray,
                            verbose = vrbs,
                        )

        if(return_best):
            best_model = load_model('best_lstm.h5')
        returnModel = model
        if(return_best):
            returnModel = best_model
        return [returnModel,min(history.history['val_loss']),len(history.history['val_loss']),history]

In [11]:
modelType = "LSTM"
trainingSetAdv, trainingSetLabels, testSet, validationSet = getReshapedAdversarialDataSet(dataSet, advDataSet, modelType)
numOfLayers = 1
numOfNeurons = 20 
[model, validatoinLoss, numOfEpochs, history] = trainLSTMAdv(numOfLayers, numOfNeurons, trainingSetAdv, trainingSetLabels, validationSet)
model.save('Trained_Model/lstm_adv.h5')

Training model.
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 200, 20)           2400      
_________________________________________________________________
dense_8 (Dense)              (None, 200, 9)            189       
_________________________________________________________________
activation (Activation)      (None, 200, 9)            0         
Total params: 2,589
Trainable params: 2,589
Non-trainable params: 0
_________________________________________________________________
Train on 11310 samples, validate on 1382 samples
Epoch 1/10000
11310/11310 [==============================] - 1s 120us/sample - loss: 0.0702 - val_loss: 0.0299
Epoch 2/10000
11310/11310 [==============================] - 0s 20us/sample - loss: 0.0589 - val_loss: 0.0291
Epoch 3/10000
11310/11310 [==============================] - 0s 18us/sample - loss: 0.0491 - val

11310/11310 [==============================] - 0s 28us/sample - loss: 0.0016 - val_loss: 0.0045
Epoch 68/10000
11310/11310 [==============================] - 0s 18us/sample - loss: 0.0016 - val_loss: 0.0044
Epoch 69/10000
11310/11310 [==============================] - 0s 20us/sample - loss: 0.0016 - val_loss: 0.0044
Epoch 70/10000
11310/11310 [==============================] - 0s 18us/sample - loss: 0.0016 - val_loss: 0.0043
Epoch 71/10000
11310/11310 [==============================] - 0s 18us/sample - loss: 0.0015 - val_loss: 0.0042
Epoch 72/10000
11310/11310 [==============================] - 0s 27us/sample - loss: 0.0015 - val_loss: 0.0041
Epoch 73/10000
11310/11310 [==============================] - 0s 18us/sample - loss: 0.0015 - val_loss: 0.0040
Epoch 74/10000
11310/11310 [==============================] - 0s 29us/sample - loss: 0.0015 - val_loss: 0.0040
Epoch 75/10000
11310/11310 [==============================] - 0s 17us/sample - loss: 0.0015 - val_loss: 0.0039
Epoch 76/10000
1

Epoch 140/10000
11310/11310 [==============================] - 0s 17us/sample - loss: 8.9509e-04 - val_loss: 0.0013
Epoch 141/10000
11310/11310 [==============================] - 0s 19us/sample - loss: 8.8966e-04 - val_loss: 0.0013
Epoch 142/10000
11310/11310 [==============================] - 0s 26us/sample - loss: 8.8437e-04 - val_loss: 0.0013
Epoch 143/10000
11310/11310 [==============================] - 0s 21us/sample - loss: 8.7914e-04 - val_loss: 0.0013
Epoch 144/10000
11310/11310 [==============================] - 0s 18us/sample - loss: 8.7398e-04 - val_loss: 0.0013
Epoch 145/10000
11310/11310 [==============================] - 0s 20us/sample - loss: 8.6890e-04 - val_loss: 0.0012
Epoch 146/10000
11310/11310 [==============================] - 0s 34us/sample - loss: 8.6392e-04 - val_loss: 0.0012
Epoch 147/10000
11310/11310 [==============================] - 0s 26us/sample - loss: 8.5901e-04 - val_loss: 0.0012
Epoch 148/10000
11310/11310 [==============================] - 0s 18us/s

11310/11310 [==============================] - 0s 18us/sample - loss: 6.5256e-04 - val_loss: 7.4391e-04
Epoch 210/10000
11310/11310 [==============================] - 0s 20us/sample - loss: 6.5026e-04 - val_loss: 7.4174e-04
Epoch 211/10000
11310/11310 [==============================] - 0s 19us/sample - loss: 6.4800e-04 - val_loss: 7.3946e-04
Epoch 212/10000
11310/11310 [==============================] - 0s 22us/sample - loss: 6.4577e-04 - val_loss: 7.3787e-04
Epoch 213/10000
11310/11310 [==============================] - 0s 18us/sample - loss: 6.4354e-04 - val_loss: 7.3602e-04
Epoch 214/10000
11310/11310 [==============================] - 0s 31us/sample - loss: 6.4135e-04 - val_loss: 7.3445e-04
Epoch 215/10000
11310/11310 [==============================] - 0s 20us/sample - loss: 6.3918e-04 - val_loss: 7.3289e-04
Epoch 216/10000
11310/11310 [==============================] - 0s 19us/sample - loss: 6.3702e-04 - val_loss: 7.3047e-04
Epoch 217/10000
11310/11310 [===========================

11310/11310 [==============================] - 0s 21us/sample - loss: 5.2798e-04 - val_loss: 6.5955e-04
Epoch 278/10000
11310/11310 [==============================] - 0s 19us/sample - loss: 5.2646e-04 - val_loss: 6.5974e-04
Epoch 279/10000
11310/11310 [==============================] - 0s 18us/sample - loss: 5.2495e-04 - val_loss: 6.5953e-04
Epoch 280/10000
11310/11310 [==============================] - 0s 18us/sample - loss: 5.2345e-04 - val_loss: 6.5867e-04
Epoch 281/10000
11310/11310 [==============================] - 0s 18us/sample - loss: 5.2201e-04 - val_loss: 6.5825e-04
Epoch 282/10000
11310/11310 [==============================] - 0s 21us/sample - loss: 5.2048e-04 - val_loss: 6.5705e-04
Epoch 283/10000
11310/11310 [==============================] - 0s 28us/sample - loss: 5.1901e-04 - val_loss: 6.5660e-04
Epoch 284/10000
11310/11310 [==============================] - 0s 22us/sample - loss: 5.1756e-04 - val_loss: 6.5626e-04
Epoch 285/10000
11310/11310 [===========================

...

11310/11310 [==============================] - 0s 21us/sample - loss: 1.4202e-04 - val_loss: 3.4209e-04
Epoch 754/10000
11310/11310 [==============================] - 0s 19us/sample - loss: 1.4159e-04 - val_loss: 3.4047e-04
Epoch 755/10000
11310/11310 [==============================] - 0s 19us/sample - loss: 1.4117e-04 - val_loss: 3.3903e-04
Epoch 756/10000
11310/11310 [==============================] - 0s 18us/sample - loss: 1.4075e-04 - val_loss: 3.3783e-04
Epoch 757/10000
11310/11310 [==============================] - 0s 19us/sample - loss: 1.4032e-04 - val_loss: 3.3663e-04
Epoch 758/10000
11310/11310 [==============================] - 0s 20us/sample - loss: 1.3990e-04 - val_loss: 3.3570e-04
Epoch 759/10000
11310/11310 [==============================] - 0s 19us/sample - loss: 1.3947e-04 - val_loss: 3.3481e-04
Epoch 760/10000
11310/11310 [==============================] - 0s 18us/sample - loss: 1.3905e-04 - val_loss: 3.3373e-04
Epoch 761/10000
11310/11310 [===========================

11310/11310 [==============================] - 0s 17us/sample - loss: 1.1516e-04 - val_loss: 2.6763e-04
Epoch 822/10000
11310/11310 [==============================] - 0s 20us/sample - loss: 1.1481e-04 - val_loss: 2.6656e-04
Epoch 823/10000
11310/11310 [==============================] - 0s 19us/sample - loss: 1.1445e-04 - val_loss: 2.6529e-04
Epoch 824/10000
11310/11310 [==============================] - 0s 18us/sample - loss: 1.1410e-04 - val_loss: 2.6422e-04
Epoch 825/10000
11310/11310 [==============================] - 0s 18us/sample - loss: 1.1375e-04 - val_loss: 2.6333e-04
Epoch 826/10000
11310/11310 [==============================] - 0s 18us/sample - loss: 1.1340e-04 - val_loss: 2.6252e-04
Epoch 827/10000
11310/11310 [==============================] - 0s 18us/sample - loss: 1.1305e-04 - val_loss: 2.6180e-04
Epoch 828/10000
11310/11310 [==============================] - 0s 24us/sample - loss: 1.1271e-04 - val_loss: 2.6075e-04
Epoch 829/10000
11310/11310 [===========================

11310/11310 [==============================] - 0s 19us/sample - loss: 9.3935e-05 - val_loss: 2.1716e-04
Epoch 890/10000
11310/11310 [==============================] - 0s 20us/sample - loss: 9.3668e-05 - val_loss: 2.1684e-04
Epoch 891/10000
11310/11310 [==============================] - 0s 29us/sample - loss: 9.3406e-05 - val_loss: 2.1639e-04
Epoch 892/10000
11310/11310 [==============================] - 0s 18us/sample - loss: 9.3143e-05 - val_loss: 2.1571e-04
Epoch 893/10000
11310/11310 [==============================] - 0s 18us/sample - loss: 9.2881e-05 - val_loss: 2.1505e-04
Epoch 894/10000
11310/11310 [==============================] - 0s 17us/sample - loss: 9.2620e-05 - val_loss: 2.1457e-04
Epoch 895/10000
11310/11310 [==============================] - 0s 19us/sample - loss: 9.2365e-05 - val_loss: 2.1420e-04
Epoch 896/10000
11310/11310 [==============================] - 0s 22us/sample - loss: 9.2113e-05 - val_loss: 2.1320e-04
Epoch 897/10000
11310/11310 [===========================

11310/11310 [==============================] - 0s 18us/sample - loss: 7.9146e-05 - val_loss: 1.9058e-04
Epoch 958/10000
11310/11310 [==============================] - 0s 18us/sample - loss: 7.8923e-05 - val_loss: 1.9075e-04
Epoch 959/10000
11310/11310 [==============================] - 0s 19us/sample - loss: 7.8938e-05 - val_loss: 1.8998e-04
Epoch 960/10000
11310/11310 [==============================] - 0s 20us/sample - loss: 7.8560e-05 - val_loss: 1.8841e-04
Epoch 961/10000
11310/11310 [==============================] - 0s 17us/sample - loss: 7.8484e-05 - val_loss: 1.8777e-04
Epoch 962/10000
11310/11310 [==============================] - 0s 17us/sample - loss: 7.8319e-05 - val_loss: 1.8733e-04
Epoch 963/10000
11310/11310 [==============================] - 0s 20us/sample - loss: 7.8083e-05 - val_loss: 1.8769e-04
Epoch 964/10000
11310/11310 [==============================] - 0s 18us/sample - loss: 7.7860e-05 - val_loss: 1.8833e-04
Epoch 965/10000
11310/11310 [===========================

11310/11310 [==============================] - 0s 19us/sample - loss: 6.8172e-05 - val_loss: 1.7382e-04
Epoch 1026/10000
11310/11310 [==============================] - 0s 18us/sample - loss: 6.8254e-05 - val_loss: 1.7322e-04
Epoch 1027/10000
11310/11310 [==============================] - 0s 19us/sample - loss: 6.7887e-05 - val_loss: 1.7258e-04
Epoch 1028/10000
11310/11310 [==============================] - 0s 17us/sample - loss: 6.7750e-05 - val_loss: 1.7242e-04
Epoch 1029/10000
11310/11310 [==============================] - 0s 24us/sample - loss: 6.7592e-05 - val_loss: 1.7246e-04
Epoch 1030/10000
11310/11310 [==============================] - 0s 18us/sample - loss: 6.7678e-05 - val_loss: 1.7419e-04
Epoch 1031/10000
11310/11310 [==============================] - 0s 28us/sample - loss: 6.7314e-05 - val_loss: 1.7155e-04
Epoch 1032/10000
11310/11310 [==============================] - 0s 17us/sample - loss: 6.7230e-05 - val_loss: 1.7108e-04
Epoch 1033/10000
11310/11310 [===================